In [ ]:
from numpy.typing import NDArray
from numpy import full, asarray
from os.path import isfile, isdir, join
from os import mkdir, scandir, cpu_count
from itertools import groupby
from cv2.typing import MatLike, Size
from cv2 import (cvtColor, imread, imwrite, resize, Canny,
  GaussianBlur, ADAPTIVE_THRESH_GAUSSIAN_C, THRESH_BINARY,
  adaptiveThreshold, COLOR_BGR2GRAY, COLOR_GRAY2BGR)
from lpmalgos import Ellipsoid, find_clusters
from shutil import rmtree

In [ ]:
BLACK, WHITE, RED = 0, 255, (0, 0, 255)
larrs = list[NDArray]

def checkValidImageFormat(extension: str):
  valid = ("bmp", "dib", "jpeg", "jpg", "jpe",
    "jp2", "png", "pnm", "avif", "pbm", "pgm",
    "ppm", "pxm", "webp", "pfm", "sr", "ras",
    "tiff", "tif", "exr", "hdr", "pic")
  if not extension in valid:
    raise TypeError("Não é uma imagem!")

def bgr2gray(entrie: MatLike):
  return cvtColor(entrie, COLOR_BGR2GRAY)

def gray2bgr(entrie: MatLike):
  return cvtColor(entrie, COLOR_GRAY2BGR)

In [ ]:
def imopen(entrie: str):
  def imopen_core(entrie: str):
    name, ext = entrie.split(".")
    checkValidImageFormat(ext)
    if isdir(name): rmtree(name)
    return name, bgr2gray(imread(entrie))

  if isfile(entrie):
    yield imopen_core(entrie)
    return
  if isdir(entrie):
    yield from (imopen_core(join(entrie, e.name))
      for e in scandir(entrie) if e.is_file())
    return

  raise FileNotFoundError(f"{entrie} ñ existe!")

In [ ]:
def cutImage(entrie: MatLike, shape: Size):
  if len(shape) != 2: raise TypeError(
    "\"shape\" não tem tamanho 2!")
  x1, x2, y1, y2 = 180, 2445, 305, 6095
  cutted = entrie[x1:x2, y1:y2]
  cutted = resize(cutted, shape[::-1])
  print("cutImage terminado!")
  return cutted

In [ ]:
def gaussProcess(entrie: MatLike, size = 19, C = 10):
  output = adaptiveThreshold(entrie, 255,
    ADAPTIVE_THRESH_GAUSSIAN_C, THRESH_BINARY, size, C)
  output = GaussianBlur(output, (11, 11), 0)
  print("gaussProcess terminado!")
  return output

In [ ]:
def cannyFilter(entrie: MatLike):
  output = Canny(entrie, 200, 255, L2gradient = True)
  print("cannyFilter terminado!")
  return output

In [ ]:
def skeletonize(entrie: MatLike, distance = 12):
  output = entrie.copy(); trans = output.T
  whites = asarray((trans == WHITE).nonzero())
  whites = groupby(whites.T, lambda x: x[0])
  for line, elems in whites:
    point2, elems = 0, [*elems]
    lenelems = len(elems)
    while True:
      point1 = point2
      (a, y0) = (a, y1) = elems[point1]
      for pos in range(point1 + 1, lenelems):
        (a, ny), point2 = elems[pos], pos
        if (ny - y0) > distance: break
        (a, y1) = (a, ny)
      if point1 == point2: break
      trans[line, y0:(y1 + 1)] = BLACK
      midpoint = ((y0 + y1) // 2)
      trans[line, midpoint] = WHITE
  print("skeletonize terminado!")
  return output

In [ ]:
def separeLines(entrie: MatLike, quant = 2000):
  def separeLines_core():
    dim1, dim2 = (entrie == WHITE).nonzero()
    data_locs = asarray((dim2, dim1, [0] * len(dim1))).T
    ani = Ellipsoid(((.3, 0, 0), (0, 2.6, 0), (0, 0, 0)))
    clusters = asarray(find_clusters(data_locs,
      ani, 20, 1, 1e-9, 1, cpu_count()))
    for cluster in set(clusters):
      dim2, dim1, a = data_locs[clusters == cluster].T
      if len(dim2) < quant: continue
      yield asarray((dim1, dim2)).T[dim2.argsort()]

  output = [*separeLines_core()]
  print("separeLines terminado!")
  return output

In [ ]:
def meanRemove(entrie: larrs):
  MIN, MAX = 550, 5
  curves: larrs = []
  basels: larrs = []
  for array in entrie:
    (A, B) = array.T
    choice = curves
    mean = A.mean(dtype = int)
    res = (abs(A - mean) > MAX)
    if res.sum() < MIN:
      choice, res = basels, ~res
      A, B = A[res], B[res]
    array = asarray((A, B)).T
    choice.append(array)
  print("meanRemove terminado!")
  return curves, basels

In [ ]:
def saveLines(lines: larrs, dname: str, dtype: str, shape: Size):
  if len(shape) != 2: raise TypeError
  pasta, preta = f"{dname}/lines", full(shape, BLACK, "u1")
  if not isdir(pasta): mkdir(pasta)
  for i, subarray in enumerate(lines):
    imagem, (dim1, dim2) = preta.copy(), subarray.T
    imagem[dim1, dim2] = WHITE
    imwrite(f"{pasta}/{dtype}{i}.png", imagem)
    with open(f"{pasta}/{dtype}{i}.txt", "w") as saida:
      saida.write(f"Tam.: {shape}\n")
      for tupla in subarray: saida.write(f"{tupla}\n")
  print("saveLines terminado!")

In [ ]:
def sobrepor(entrie: MatLike, listas: larrs):
  output = gray2bgr(entrie)
  dim1, dim2 = zip(*(e for a in listas for e in a))
  output[dim1, dim2] = RED
  print("sobrepor terminado!")
  return output

In [ ]:
def components_with_t(base: float, mm_const: float, mm_diff: NDArray,
                      q_comp: float, t_comp: float):
  return (base + (mm_const * mm_diff) - (q_comp * t_comp))

def components_wout_t(base: float, mm_const: float, mm_diff: NDArray):
  return components_with_t(base, mm_const, mm_diff, 0, 0)

d_component = t_component = components_wout_t
h_component = z_component = components_with_t

In [ ]:
def differs(curves: larrs, basels: larrs,
            h_base: float, h_const: float, hq_comp: float,
            d_base: float, d_const: float,
            z_base: float, z_const: float, zq_comp: float,
            t_comp: float,
            dir_name: str, px_to_mm: float):
  pasta, (h, d, z) = f"{dir_name}/diffs", curves
  if not isdir(pasta): mkdir(pasta)
  for b, basel in enumerate(basels):
    meanbase = basel.T[0].mean().round().astype(int)
    with open(fr"{pasta}/h{b}.txt", "w") as o:
      o.writelines(map(str, h_component(h_base, h_const, ((meanbase - h.T[0]) * px_to_mm), hq_comp, t_comp)))
    with open(fr"{pasta}/d{b}.txt", "w") as o:
      o.writelines(map(str, d_component(d_base, d_const, ((meanbase - d.T[0]) * px_to_mm))))
    with open(fr"{pasta}/z{b}.txt", "w") as o:
      o.writelines(map(str, z_component(z_base, z_const, ((meanbase - z.T[0]) * px_to_mm), zq_comp, t_comp)))
  print("differs terminado!")

In [ ]:
shape, const = (1800, 4590), (200 / 4590)
for name, opened in imopen("Imagens"):
  print(f"Imagem atual: {name}")
  if not isdir(name): mkdir(name)

  cutted = cutImage(opened, shape)
  imwrite(f"{name}/cutted.png", cutted)
  gauss = gaussProcess(cutted)
  imwrite(f"{name}/gauss.png", gauss)
  canny = cannyFilter(gauss)
  imwrite(f"{name}/canny.png", canny)

  skeleton = skeletonize(canny)
  imwrite(f"{name}/skeleton.png", skeleton)

  separed = separeLines(skeleton)
  curves, basels = meanRemove(separed)

  saveLines(curves, name, "curve", shape)
  saveLines(basels, name, "basel", shape)
  differs(curves, basels, name, const)

  sobre1 = sobrepor(skeleton, curves)
  sobre2 = sobrepor(canny, curves)
  imwrite(f"{name}/sobre1.png", sobre1)
  imwrite(f"{name}/sobre2.png", sobre2)
  print("\n----------------------\n")

print("Processo Terminado!")